In [1]:
# Problem:
# pos and dpos don't reset between gradient descent iterations damn




# Main resources
# https://github.com/taichi-dev/taichi/blob/master/python/taichi/examples/autodiff/minimization.py

from datetime import datetime 

start_time = datetime.now() 
import sys
sys.tracebacklimit=0


import random

import taichi as ti
import time 

# ti.init(arch=ti.gpu)
# ti.init(arch=ti.cpu,cpu_max_num_threads=1,debug=True)
ti.init(arch=ti.cpu)
# Number of segments in the rope
N = 10
# Time steps the simulation goes for
T = 15

GOAL = [120,60]


u = ti.field(ti.f32, shape=())
pos = ti.Vector.field(n=2,dtype=ti.f32, shape=(T,N))
d_pos = ti.Vector.field(n=2,dtype=ti.f32, shape=(T,N))




L = ti.field(dtype=ti.f32, shape=())
ti.root.lazy_grad()


@ti.func
def F_spring(vec):  # Force of spring, see https://youtu.be/FcnvwtyxLds?t=612
    spring_k=1
    l=1
    return spring_k*( vec.norm(1e-6) - l) * vec.normalized(1e-6) 

@ti.kernel
def simulate():
#   pos[0] (initial conditions) is already done
    for t in ti.static(range(1,T)):
        for i in ti.static(range(1,N)):
#         verlet integration on all rope segments
#                 d_pos[t,i] += pos[t-1,i]-pos[t-2,i] + ti.Vector([0,10])
            d_pos[t,i] += d_pos[t-1,i] + ti.Vector([0,10])

        ######     Constraints

        
        d_pos[t,0] += 100 * ti.Vector([ti.math.cos(u[None]),ti.math.sin(u[None])])
        
        for i in ti.static(range(1,N-1)):
            v1 = F_spring(pos[t-1,i-1] - pos[t-1,i]) * 0.5
            v2 = F_spring(pos[t-1,i+1] - pos[t-1,i]) * 0.5
            d_pos[t,i] += (v1 + v2)
#             print('i =', i)
        
#         REMEMBER THE LARGER ONE (N-1) IS ON THE RIGHT
        v1 = F_spring(pos[t-1,(N-1)-1] - pos[t-1,N-1]) * 0.5
        d_pos[t,N-1] += v1
        
        for i in ti.static(range(N)):
            pos[t,i] += pos[t-1,i] + d_pos[t,i]

        
@ti.kernel
def compute_loss():
    for t in ti.static(range(T)):
        L[None] += t * (pos[t,N-1] - ti.Vector(GOAL)).norm(1e-6)
#     L[None] += (pos[T-1,N-1] - ti.Vector([120.0,60.0])).norm(1e-6)
        

@ti.kernel
def gradient_descent():
#     for i in ti.static(range(2)):
    # Time steps the simulation goes for:
    u[None] -= 10 * u.grad[None]
#         print(u.grad[None][d])

@ti.kernel
def reset():
#     the -1 and +1 make it so that the initial conditions don't change
    for t, n in ti.ndrange(T - 1, N):
        pos[t + 1, n] = ti.Vector([0.0, 0.0])
        d_pos[t + 1 , n] = ti.Vector([0.0, 0.0])
        
u[None] = 0
def main():
    # Initialize positions of rope segments
    for i in range(N):
#         pos[0,i] = [i * random.random() * 100,i * random.random() * 100]
        pos[0,i] = [i * 10,i * 10]
        d_pos[0,i] = [0,0]

    
    # Optimize with m gradient descent iterations
    for k in range(100):
        with ti.ad.Tape(loss=L):
            reset()
            simulate()
            compute_loss()
            print('Loss =', L[None])
        gradient_descent()
        
#         u[None][0] = sorted((-500, u[None][0], 500))[1]
#         u[None][1] = sorted((-500, u[None][1], 500))[1]

        
        
    time_elapsed = datetime.now() - start_time 
    print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))


if __name__ == '__main__':
    main()
    


[Taichi] version 1.3.0, llvm 15.0.1, commit 0f25b95e, win, python 3.9.12
[Taichi] Starting on arch=x64
Loss = 57087.875
Loss = 54875.84375
Loss = 58285.83203125
Loss = 59501.5390625
Loss = 59357.6796875
Loss = 59513.31640625
Loss = 59083.484375
Loss = 55259.01953125
Loss = 59263.17578125
Loss = 58649.66796875
Loss = 59424.3984375
Loss = 59401.7265625
Loss = 56766.03125
Loss = 55531.71875
Loss = 55266.26953125
Loss = 58629.328125
Loss = 55161.48046875
Loss = 57438.95703125
Loss = 59030.60546875
Loss = 57303.0234375
Loss = 59170.25
Loss = 55918.00390625
Loss = 55011.140625
Loss = 54944.1015625
Loss = 55164.671875
Loss = 56733.71875
Loss = 59394.50390625
Loss = 54891.859375
Loss = 56339.703125
Loss = 55336.62109375
Loss = 59466.7265625
Loss = 55730.4609375
Loss = 56194.0859375
Loss = 58434.7109375
Loss = 56771.765625
Loss = 57910.5
Loss = 57858.00390625
Loss = 54868.0546875
Loss = 58171.609375
Loss = 55869.9921875
Loss = 54934.9609375
Loss = 59502.0546875
Loss = 59440.984375
Loss = 54872.

In [2]:
list(range(1,N-1))

[1, 2, 3, 4, 5, 6, 7, 8]

In [3]:
# print(pos)
print(u)
print(u.grad[None])


-32520.6
-2030.4530029296875


In [4]:
print(d_pos)

[[[  0.           0.        ]
  [  0.           0.        ]
  [  0.           0.        ]
  [  0.           0.        ]
  [  0.           0.        ]
  [  0.           0.        ]
  [  0.           0.        ]
  [  0.           0.        ]
  [  0.           0.        ]
  [  0.           0.        ]]

 [[-73.0622     -68.27821   ]
  [  0.          10.        ]
  [  0.          10.        ]
  [  0.          10.        ]
  [  0.          10.        ]
  [  0.          10.        ]
  [  0.          10.        ]
  [  0.          10.        ]
  [  0.          10.        ]
  [ -4.646446     5.353554  ]]

 [[-73.0622     -68.27821   ]
  [-36.542027   -19.128513  ]
  [  0.          20.        ]
  [  0.          20.        ]
  [  0.          20.        ]
  [  0.          20.        ]
  [  0.          20.        ]
  [  0.          20.        ]
  [ -2.3232229   17.676777  ]
  [ -6.9696693   13.030331  ]]

 [[-73.0622     -68.27821   ]
  [-73.077866   -53.264004  ]
  [-18.280697    10.445169  ]
  [ 

In [5]:
print(pos)

[[[ 0.00000000e+00  0.00000000e+00]
  [ 1.00000000e+01  1.00000000e+01]
  [ 2.00000000e+01  2.00000000e+01]
  [ 3.00000000e+01  3.00000000e+01]
  [ 4.00000000e+01  4.00000000e+01]
  [ 5.00000000e+01  5.00000000e+01]
  [ 6.00000000e+01  6.00000000e+01]
  [ 7.00000000e+01  7.00000000e+01]
  [ 8.00000000e+01  8.00000000e+01]
  [ 9.00000000e+01  9.00000000e+01]]

 [[-7.30622025e+01 -6.82782135e+01]
  [ 1.00000000e+01  2.00000000e+01]
  [ 2.00000000e+01  3.00000000e+01]
  [ 3.00000000e+01  4.00000000e+01]
  [ 4.00000000e+01  5.00000000e+01]
  [ 5.00000000e+01  6.00000000e+01]
  [ 6.00000000e+01  7.00000000e+01]
  [ 7.00000000e+01  8.00000000e+01]
  [ 8.00000000e+01  9.00000000e+01]
  [ 8.53535538e+01  9.53535538e+01]]

 [[-1.46124405e+02 -1.36556427e+02]
  [-2.65420265e+01  8.71486664e-01]
  [ 2.00000000e+01  5.00000000e+01]
  [ 3.00000000e+01  6.00000000e+01]
  [ 4.00000000e+01  7.00000000e+01]
  [ 5.00000000e+01  8.00000000e+01]
  [ 6.00000000e+01  9.00000000e+01]
  [ 7.00000000e+01  1.00

In [6]:
import pygame
import time
import math




def rendered_pos(p):
    magnification = 1
    return [p[0]/magnification + 500,p[1]/magnification + 500]



def render(t):
    
    list_pos = [ rendered_pos(pos[t,i]) for i in range(N) ]
#     print(list_pos)
    
    window.fill((255, 255, 255))
    
    
    for pair in list(zip(list_pos, list_pos[1:])):
        pygame.draw.line(window, (0,0,0), pair[0], pair[1],width=5)
    for p in list_pos[:-1]:
        pygame.draw.circle(window, (0, 255, 0),list(p), 10, 0)
    pygame.draw.circle(window, (255, 0, 0),list(list_pos[N-1]), 10, 0)
    pygame.draw.circle(window,(0,0,255),rendered_pos(GOAL), 10, 0)
    
    
    u_direction_pos = [list_pos[0][0] + ti.math.cos(u[None]) * 40,list_pos[0][1] + ti.math.sin(u[None]) * 40]
    pygame.draw.circle(window,(0,255,255),u_direction_pos, 10, 0)
    
#     pygame.draw.circle(window, (255, 0, 0),list(anchor), 5, 0)
    # Draws the surface object to the screen.
    img = font.render(str(t), True,(0, 0, 0))
    window.blit(img, (20, 20))
    
    pygame.display.update()

pygame.init()
font = pygame.font.SysFont(None, 40)
window = pygame.display.set_mode((1000, 1000))
run = True
while run:
    for t in range(T):
#         print(t)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        render(t)
        time.sleep(0.0167 * 50)

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 